<h1><center>Laboratorio 8: ¿Superhéroe o Villano? 🦸</center></h1>

<center><strong>MDS7202: Laboratorio de Programación Científica para Ciencia de Datos</strong></center>

### Cuerpo Docente:

- Profesor: Pablo Badilla
- Auxiliar: Ignacio Meza D.
- Ayudante: Patricio Ortiz.

### Equipo: SUPER IMPORTANTE - notebooks sin nombre no serán revisados

- Nombre de alumno 1: Sebastián Tinoco
- Nombre de alumno 2:


### **Link de repositorio de GitHub:** `https://github.com/sebatinoco/MDS7202/tree/main/lab8`

## Temas a tratar

- Codificación de texto usando Bag of Words.
- Búsqueda del modelo óptimo de clasificación usando `GridSearch`
- Uso de pipelines.

## Reglas:

- Fecha de entrega: 23/06/2022
- **Grupos de 2 personas**
- **Ausentes** deberán realizar la actividad solos. 
- Cualquier duda fuera del horario de clases al foro. Mensajes al equipo docente serán respondidos por este medio.
- Prohibidas las copias. 
- Pueden usar cualquer matrial del curso que estimen conveniente.

### Objetivos principales del laboratorio

- Obtener caracteristicas a partir de texto usando `CountVectorizer`.
- Fijar un pipeline con un modelo base que luego se irá optimizando.
- Comprender como realizar una búsqueda de grilla sobre un conjunto de clasificadores e hiperparámetros usando `GridSearch`.

El laboratorio deberá ser desarrollado sin el uso indiscriminado de iteradores nativos de python (aka "for", "while"). La idea es que aprendan a exprimir al máximo las funciones optimizadas que nos entrega `pandas`, las cuales vale mencionar, son bastante más eficientes que los iteradores nativos sobre DataFrames.

#Importamos librerias utiles 😸

In [1]:
# Librería Core del lab.
import numpy as np
import pandas as pd
from sklearn.pipeline import Pipeline

from sklearn.model_selection import train_test_split 

# Pre-procesamiento
from sklearn.feature_selection import SelectPercentile, f_classif
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_extraction.text import CountVectorizer

# Clasifación
from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

# Metricas de evaluación
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import cohen_kappa_score

# Librería para plotear
!pip install --upgrade plotly
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go

# Proyecciones en baja dimensionalidad: UMAP
!pip install umap-learn

# Librería para NLP
!pip install nltk
import nltk
from nltk.corpus import stopwords
from nltk import word_tokenize  
from nltk.stem import PorterStemmer
nltk.download('stopwords')
nltk.download('punkt')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

# 1. ¿Quien es Bat Cow?

<p align="center">
  <img src="https://i.imgur.com/D9f1RHy.jpg" width="350">
</p>

En vez de estar desarrollando las evaluaciones correspondientes a su curso, su profesor de catedra y su auxiliar discuten acerca la alineación (héroe o villano) del personaje de ficción Bat-Cow. 

El cuerpo docente, no logra ponerse de acuerdo si el personaje es bueno, neutral o malo: el auxiliar plantea que Bat-cow posee una siniestra mirada, intrigante pero común característica de los personajes malvados. 
Por otra parte, extendiendo las ideas de Rousseau, el profesor plantea que tal como los humanos no nacen malos, no existe motivo por el cual una vaca con superpoderes deba serlo.

Sin embargo, ambos concuerdan que es difícil estimar la alineación solo usando los atributos físicos, por lo que creen el análisis debe ser complementado aún más antes de comunicarle los resultados a su estudiantado. Buscando más información, ambos sujetos se percatan de la existencia de un excelente antecedente para estimar la alineación: la historia personal de cada superhéroe o villano.

Es por esto le solicitan que construya y optimice un clasificador basado en texto el cual analice la alineación de cada personaje basado en su historia personal.

Para este laboratorio deben trabajar con los datos `df_comics.csv` y `comics_no_label.csv` subidos a u-cursos. El primero es un conjunto de datos que les servirá para entrenar un modelo de clasificación, mientras que el segundo es un dataset con personajes de ficción no etiquetados a predecir (sí, aquí está la misteriosa Batcow).

Para comenzar cargue los dataset señalados y visualice a través de un head los atributos que poseen cada uno de los dataset.


In [2]:
# Si usted está utilizando Colabolatory le puede ser útil este código para cargar los archivos.
try:
    from google.colab import drive
    drive.mount("/content/drive")
    path = '/content/drive/MyDrive/MDS/Labs/Lab 8/'
except: 
    print('Ignorando conexión drive-colab')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
df_comics = pd.read_csv(path + 'df_comics.csv')
df_comics_no_label = pd.read_csv(path + 'comics_no_label.csv')
df_comics = df_comics.dropna(subset=['history_text']) # eliminar ejemplos sin historia

In [4]:
# queda a labor de su equipo hacer el análisis exploratorio
df_comics.head()

,Unnamed: 0,name,real_name,full_name,overall_score,history_text,powers_text,intelligence_score,strength_score,speed_score,...,has_flight,has_accelerated_healing,has_weapons_master,has_intelligence,has_reflexes,has_super_speed,has_durability,has_stamina,has_agility,has_super_strength
0,0,3-D Man,"Delroy Garrett, Jr.","Delroy Garrett, Jr.",6,"Delroy Garrett, Jr. grew up to become a track ...",NaN,85,30,60,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
1,2,A-Bomb,Richard Milhouse Jones,Richard Milhouse Jones,20,"Richard ""Rick"" Jones was orphaned at a young ...","On rare occasions, and through unusual circu...",80,100,80,...,0.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0
2,3,Aa,Aa,NaN,12,Aa is one of the more passive members of the P...,NaN,80,50,55,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,4,Aaron Cash,Aaron Cash,Aaron Cash,5,Aaron Cash is the head of security at Arkham A...,NaN,80,10,25,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,5,Aayla Secura,Aayla Secura,NaN,8,ayla Secura was a Rutian Twi'lek Jedi Knight (...,NaN,90,40,45,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


## 1.1 Obtención de Features y Bag of Words

<p align="center">
  <img src="https://media0.giphy.com/media/eIUpSyzwGp0YhAMTKr/200.gif" width="300">
</p>

Primero que todo, deben obtener un vector de características del atributo `history_text`, utilizando `Bag of Words`. En este atributo se presenta una breve descripción de la historia de cada uno de los personajes de ficción presentes en el dataset. 

Pero... antes de empezar, ¿Que es `Bag of Words`?...

`Bag of Words` es un modelo de conteo utilizado en Procesamiento de Lenguaje Natural (NLP) que tiene como objetivo generar una representación vectorial (vector de características en nuestro cas) para cada documento a través del conteo de las palabras que contienen. 

La siguiente figura muestra un ejemplo de `Bag of Words` en acción:

<p align="center">
  <img src="https://user.oc-static.com/upload/2020/10/23/16034397439042_surfin%20bird%20bow.png" width="500">
</p>

Como pueden ver, el modelo de `Bag of Words` no resulta tan complicado, ¿pero cómo lo aplicamos en python?. 

Como podrán darse cuenta del ejemplo anterior, para facilitar el conteo será necesario transformar cada uno de los documentos en vectores, donde cada una de las posiciones posee un carácter. Este proceso es conocido como **tokenización** y lo podemos realizar de la siguiente forma:

In [5]:
docs = ['The teacher rocks like a good rock & roll',
             'the rock is the best actor in the world']


docs_tokenizados = [word_tokenize(doc)  for doc in docs]
docs_tokenizados

[['The', 'teacher', 'rocks', 'like', 'a', 'good', 'rock', '&', 'roll'],
 ['the', 'rock', 'is', 'the', 'best', 'actor', 'in', 'the', 'world']]

Podemos mejorar un poco más el proceso de tokenización agregando 

- Stemming:  Definimos Stemming como un algoritmo basado en reglas que transforma las palabras a una forma general. Un ejemplo de stemming, es el siguiente:
- Eliminación de Stopwords: Eliminación de palabras muy frecuentes que entorpecen la clasificación (por ejemplo, el, la los, la, etc...)

<p align="center">
  <img src="https://devopedia.org/images/article/218/8583.1569386710.png" width="300">
</p>


In [6]:
# Definimos algunas stopword que queremos que sean eliminadas
import nltk
nltk.download('stopwords')

from nltk.corpus import stopwords

stop_words = stopwords.words('spanish')

# Definimos un tokenizador con Stemming
class StemmerTokenizer:
    def __init__(self):
        self.ps = PorterStemmer()
    def __call__(self, doc):
        doc_tok = word_tokenize(doc)
        doc_tok = [t for t in doc_tok if t not in stop_words]
        return [self.ps.stem(t) for t in doc_tok]

# Inicializamos tokenizador
tokenizador = StemmerTokenizer()

# Creamos algunos documentos
docs = ['The teacher rocks like a good rock & roll',
        'the rock is the best actor in the world',
        'New York is a beautiful city']

# Obtenemos el token del primer documento
[tokenizador(doc) for doc in docs]

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


[['the', 'teacher', 'rock', 'like', 'good', 'rock', '&', 'roll'],
 ['the', 'rock', 'is', 'the', 'best', 'actor', 'in', 'the', 'world'],
 ['new', 'york', 'is', 'beauti', 'citi']]

In [7]:
# Comparación con el caso anterior
docs_tokenizados = [word_tokenize(doc) for doc in docs]
docs_tokenizados

[['The', 'teacher', 'rocks', 'like', 'a', 'good', 'rock', '&', 'roll'],
 ['the', 'rock', 'is', 'the', 'best', 'actor', 'in', 'the', 'world'],
 ['New', 'York', 'is', 'a', 'beautiful', 'city']]

#### Al Estilo Scikit

Scikit implementa `bag of words` a través de la clase `CountVectorizer()` la cual contiene muchas opciones para mejorar la tokenización.

In [8]:
bow = CountVectorizer(tokenizer= StemmerTokenizer())
df = bow.fit_transform(docs)

pd.DataFrame(df.toarray(), columns=bow.get_feature_names_out())

,&,actor,beauti,best,citi,good,in,is,like,new,rock,roll,teacher,the,world,york
0,1,0,0,0,0,1,0,0,1,0,2,1,1,1,0,0
1,0,1,0,1,0,0,1,1,0,0,1,0,0,3,1,0
2,0,0,1,0,1,0,0,1,0,1,0,0,0,0,0,1


Una de las cosas más interesantes que provee son el use de n-gramas, los cuales, en palabras simples, son conjuntos de n-palabras que se concatenan entre si y que se consideran como tokens separados. 

Pensemos en `Nueva York`. Cuando se tokeniza Nueva York, se generan dos tokens independientes que a simple vista no tienen relación: `Nueva` `York`.
Al usar n-gramas (en un rango min=1,max=2) , generamos tanto `Nueva` y `York` como también `Nueva York` como un token independiente.

In [9]:
bow = CountVectorizer(tokenizer= StemmerTokenizer(), ngram_range=(1,2))
df = bow.fit_transform(docs)

pd.DataFrame(df.toarray(), columns=bow.get_feature_names_out())

,&,& roll,actor,actor in,beauti,beauti citi,best,best actor,citi,good,...,teacher,teacher rock,the,the best,the rock,the teacher,the world,world,york,york is
0,1,1,0,0,0,0,0,0,0,1,...,1,1,1,0,0,1,0,0,0,0
1,0,0,1,1,0,0,1,1,0,0,...,0,0,3,1,1,0,1,1,0,0
2,0,0,0,0,1,1,0,0,1,0,...,0,0,0,0,0,0,0,0,1,1


De los resultados, podemos ver que generamos vectores de conteo para cada una de las palabras que conforman el corpus.  Un punto extra que se agrega en esta obtención de frecuencias son los bigramas, que básicamente son el conjunto de palabras de tamaño de aparecen juntas en el texto.

## Codificando los Super{heroes, villanos}  [0.5 Puntos]

<p align="center">
  <img src="https://c.tenor.com/LkQzw7k5DV4AAAAd/anime-hacking.gif" width="300">
</p>

Conociendo ahora que es el proceso de `bag of words`, aplique este modelo de obtención de caracteristicas de la siguiente forma en un pipeline:

- Utilice el tokenizador entregado.
- Obtenga caracteristicas de los unigramas y bigramas del texto (tal como el ejemplo).

```python
bog = CountVectorizer(tokenizer= StemmerTokenizer(),`
                      ngram_range=(1,2) # Este punto es opcional y es para generar bigramas
                      )
```

Finalmente, aplique `MinMaxScaler()` sobre `atributos_de_interes` y concatene el valor obtenido con el matriz de caracteristicas obtenidas con bag of words.

```python
atributos_de_interes = ['intelligence_score', 'strength_score', 'speed_score', 'durability_score', 'power_score', 'combat_score']
```

No es necesario que obtenga un dataframe en concreto con las características solicitadas. Se le recomienda generar un `ColumnTransformer()` para aplicar las transformaciones solicitadas en un pipeline.

**To-Do:**
- [ ] Obtener a traves de Bag of Words (`CountVectorizer`) caracteristicas del resumen de historia de cada personaje.
- [ ] Aplicar `MinMaxScaler` sobre los atributos de interes.

**Respuesta:**

In [10]:
# instanciamos objeto Bag of Words
bow = CountVectorizer(tokenizer= StemmerTokenizer(),
                      ngram_range=(1,2) # Este punto es opcional y es para generar bigramas
                      )

In [11]:
df_bow = bow.fit_transform(df_comics['history_text']) # obtenemos features bow
df_bow = pd.DataFrame(df_bow.toarray(), columns = bow.get_feature_names_out()) # guardamos en dataframe
df_bow

,!,! !,! '',! (,! ),"! ,",! --,! .,! ``,! alison,...,�kick� .,�kick� and,�polic,�polic man�,�white,�white room�,�ǣmortal,�ǣmortal instruments���,��wors,"��wors ,"
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1280,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1281,1,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1282,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1283,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Procedemos ahora a aplicar el escalado de los atributos de interés por medio de `MinMaxScaler`.

In [49]:
atributos_de_interes = ['intelligence_score', 'strength_score', 'speed_score', 'durability_score', 'power_score', 'combat_score']

scaler = MinMaxScaler()

interesting_features = pd.DataFrame(scaler.fit_transform(df_comics[atributos_de_interes]), columns = atributos_de_interes)
interesting_features

,intelligence_score,strength_score,speed_score,durability_score,power_score,combat_score
0,0.85,0.3,0.60,0.60,0.40,0.70
1,0.80,1.0,0.80,1.00,1.00,0.80
2,0.80,0.5,0.55,0.45,1.00,0.55
3,0.80,0.1,0.25,0.40,0.30,0.50
4,0.90,0.4,0.45,0.55,0.55,0.85
...,...,...,...,...,...,...
1280,0.90,0.1,0.25,0.30,1.00,0.55
1281,0.80,1.0,1.00,1.00,1.00,0.80
1282,0.95,0.5,1.00,0.75,1.00,0.80
1283,0.75,0.1,1.00,0.30,1.00,0.30


Finalmente, podemos ver como se verían los datos si los uniéramos en un dataframe

In [43]:
df_bow = pd.concat([df_bow, interesting_features], axis = 1) # concat de features
df_bow

,!,! !,! '',! (,! ),"! ,",! --,! .,! ``,! alison,...,2,3,4,5,intelligence_score,strength_score,speed_score,durability_score,power_score,combat_score
0,0,0,0,0,0,0,0,0,0,0,...,0.60,0.60,0.40,0.70,0.85,0.3,0.60,0.60,0.40,0.70
1,0,0,0,0,0,0,0,0,0,0,...,0.80,1.00,1.00,0.80,0.80,1.0,0.80,1.00,1.00,0.80
2,0,0,0,0,0,0,0,0,0,0,...,0.55,0.45,1.00,0.55,0.80,0.5,0.55,0.45,1.00,0.55
3,0,0,0,0,0,0,0,0,0,0,...,0.25,0.40,0.30,0.50,0.80,0.1,0.25,0.40,0.30,0.50
4,0,0,0,0,0,0,0,0,0,0,...,0.45,0.55,0.55,0.85,0.90,0.4,0.45,0.55,0.55,0.85
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1280,0,0,0,0,0,0,0,0,0,0,...,0.25,0.30,1.00,0.55,0.90,0.1,0.25,0.30,1.00,0.55
1281,1,0,1,0,0,0,0,0,0,0,...,1.00,1.00,1.00,0.80,0.80,1.0,1.00,1.00,1.00,0.80
1282,0,0,0,0,0,0,0,0,0,0,...,1.00,0.75,1.00,0.80,0.95,0.5,1.00,0.75,1.00,0.80
1283,0,0,0,0,0,0,0,0,0,0,...,1.00,0.30,1.00,0.30,0.75,0.1,1.00,0.30,1.00,0.30


Finalmente, podemos condensar todo lo anterior en un `ColumnTransformer`

In [14]:
ct = ColumnTransformer([
                   ('bow', CountVectorizer(tokenizer= StemmerTokenizer(), ngram_range=(1,2)), 'history_text'),
                   ('scaler', MinMaxScaler(), atributos_de_interes),
])

## 1.2 Diseño de Baseline y  Primer Entrenamiento  [1 Puntos]


<p align="center">
  <img src="https://pa1.narvii.com/6374/9eaec1b7bf9157334151452a669516f9a78b954c_hq.gif" width="300">
</p>



Genere un Pipeline con las caracteristicas solicitadas en la sección 1.1, un selector de mejores features `SelectPercentile` con métrica `f_classif` y percentile=90 y un clasificador `MultinomialNB()` por defecto.

Luego, separe el conjunto de datos en un conjunto de entrenamiento y prueba, donde las etiquetas estará dado por el atributo `alignment`. 

Entrene el modelo y reporte el desempeño con un `classification_report`. ¿ Nos recomendaría predecir la alineación de BatCow con este clasificador?.

Finalmente, compare el modelo entrenado con un modelo Dummy estratificado y responda: ¿El clasificador entrenado es mejor que el dummy que entrega respuestas al azar?

**To-do:**
- [ ] Realizar un pipeline con las caracteristicas solicitadas en 1.1, ejecutar holdout y aplicar un clasificador `MultinomialNB()`.
- [ ] Entrenar el pipeline, calcular el `classification_report` asociado y comentar los resultados.
- [ ] Entrenar un `DummyClassifier` con estrategia `statified`, calcular el `classification_report` asociado y comentar que implican los scores obtenidos en comparación con los resultados del baseline.

**Respuesta:**

In [15]:
target = 'alignment' # target a predecir
X = df_comics.drop(columns = target).copy() # dropeamos target para evitar fuga de información
y = df_comics[target] # vector a predecir

# separación de datos
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = .33, stratify = y, random_state = 3380)

In [16]:
pipeline = Pipeline([
                     ('vectorizer', ct), # column transformer creado anteriormente
                     ('selection', SelectPercentile(f_classif, percentile = 90)), # seleccionamos top 90%
                     ('clf', MultinomialNB()) # clasificador
                     ])

pipeline.fit(X_train, y_train) # fit
y_pred = pipeline.predict(X_test) # predict
print(classification_report(y_test, y_pred)) # classification report

              precision    recall  f1-score   support

         Bad       0.68      0.09      0.16       142
        Good       0.60      0.99      0.75       246
     Neutral       1.00      0.03      0.05        37

    accuracy                           0.60       425
   macro avg       0.76      0.37      0.32       425
weighted avg       0.66      0.60      0.49       425



En vista de los resultados, observamos que es un modelo que genera predicciones cargadas a la clase Good. Por lo tanto, se debería mejorar el ajuste a los datos para predecir `Batcow`

In [17]:
from sklearn.dummy import DummyClassifier

dummy_pipeline = Pipeline([
                     ('vectorizer', ct), # column transformer creado anteriormente
                     ('selection', SelectPercentile(f_classif, percentile = 90)), # seleccionamos top 90%
                     ('clf', DummyClassifier(strategy = 'stratified')) # clasificador
                     ])

dummy_pipeline.fit(X_train, y_train) # fit
y_pred = dummy_pipeline.predict(X_test) # predict 
print(classification_report(y_test, y_pred)) # classification report

              precision    recall  f1-score   support

         Bad       0.33      0.34      0.33       142
        Good       0.59      0.57      0.58       246
     Neutral       0.10      0.11      0.11        37

    accuracy                           0.45       425
   macro avg       0.34      0.34      0.34       425
weighted avg       0.46      0.45      0.45       425



En vista de los resultados, es posible concluir que, si bien el `MultinomialNB` obtiene un **Accuracy** mayor al `DummyClassifier`, este lo hace a través generar predicciones que son en su mayoría pertenecientes a la clase mayoritaria (Good), yendo en desmedro de las clases minoritarias (lo cual se puede ver en el **Recall** de Bad y Neutral). Esto último ocurre pues el dataset se encuentra desbalanceado y por lo tanto los modelos están sesgados a generar predicciones en favor de la clase mayoritaria. Por otro lado, el `DummyClassifier` obtiene un **Accuracy** global menor pero un **Recall** de las clases minoritarias mayor. 

Teniendo todo lo anterior en consideración, se concluye que el modelo `MultinomialNB` es levemente mejor al `DummyClassifier`, pues es incapaz de predecir las clases con menor frecuencia.

## 1.3 Busqueda del Mejor Modelo con Grid Search [4 Puntos]

<p align="center">
  <img src="https://media1.tenor.com/images/70fdfeea52a8e2e4505498c230a0d2f9/tenor.gif?itemid=5134219" width="250">
</p>

No conformes con el rendimiento obtenido en la sección 1.2, el cuerpo docente les pide que realicen un **`HalvingGridSearchCV`** con diferentes parámetros para mejorar el rendimiento de la clasificación. Para esto, se le solicita que defina:

- Tres clasificadores distintos en donde varie sus parámetros. Considere usar modelos clásicos como también los basados en ensamblaje.
- Modificar `n-gram` range del `CountVectorizer` probando `(1,1), (1,2) y (1,3)`. Examinar también los otros parámetros de CountVectorizer como por ejemplo `max_df`, `min_df`, etc... ([Documentación aquí](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html))
- Seleccionar las columnas que contribuyen con la mayor información para la clasificación con `SelectPercentile` en los percentiles `[20, 40, 60, 80]` (puede usar la métrica que usted quiera).
- Reporte la mejor combinación encontrada y justifique por qué cree que es la mejor según el clasificador usado, la cantidad de columnas seleccionadas y los parámetros de CountVectorizer seleccionados por GridSearch.

A continuación, un ejemplo de parametros para GridSearch para una búsqueda de 3 clasificadores distintos:

```python
params = [
       # clasificador 1 + hiperparámetros
       {'clf': classificator1(),
        'clf__penalty': ['ovr'],
       # clasificador 1 + hiperparámetros    
       {'clf': classificator2(),
        'clf__n_estimators': [200]},
       # clasificador 1 + hiperparámetros
       {'clf': classificator3(),
        ...
       }
       ]
```

**Nota 1**: Puede ver los parámetros modificables aplicando el método get_params() sobre su pipeline. Ver la clase de GridSearch para mayor información sobre la sintáxis de las grillas.

**Nota 2**: Recuerde inicializar los clasificadores con un random state definido.

**Nota 3**: Puede usar en `HalvingGridSearchCV` el parámetro `verbose=10` para ver que GridSearch le indique el estado de su ejecución.

**Nota 3:** El GridSearch puede tomar tiempos de búsqueda exorbitantes, por lo que se le recomienda no agrandar mucho el espacio de búsqueda, dejar corriendo el código y tomarse un tecito.

**Respuesta:**

Los clasificadores a probar serán `RandomForestClassifier`, `SVC` y `LogisticRegression`. Para RandomForest se modificará el parámetro `n_estimators`, para SVC el parámetro `kernel` y para LogisticRegression el parámetro `penalty`. Por otro lado, se probarán modificaciones al parámetro `min_df` y `binary` del CountVectorizer. Finalmente, se probarán diferentes combinaciones del `Pipeline` como fue indicado por el enunciado.

In [18]:
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression

param_grid = [
              # Random Forest
              {'vectorizer__bow__ngram_range': [(1,1), (1,2), (1,3)],
               'vectorizer__bow__min_df': [.33, 1],
               'vectorizer__bow__binary': [True, False],
               'selection__percentile': [20, 40, 60, 80],
               'clf': [RandomForestClassifier(random_state = 3380)],
               'clf__n_estimators': [100, 500]},
              # SVC
              {'vectorizer__bow__ngram_range': [(1,1), (1,2), (1,3)],
               'vectorizer__bow__min_df': [.33, 1],
               'vectorizer__bow__binary': [True, False],
               'selection__percentile': [20, 40, 60, 80],
               'clf': [SVC(random_state = 3380)],
               'clf__kernel': ['linear', 'poly', 'rbf']},
              # LogisticRegression
              {'vectorizer__bow__ngram_range': [(1,1), (1,2), (1,3)],
               'vectorizer__bow__min_df': [.33, 1],
               'vectorizer__bow__binary': [True, False],
               'selection__percentile': [20, 40, 60, 80],
               'clf': [LogisticRegression(random_state = 3380)],
               'clf__penalty': ['none', 'l2']},
]

In [19]:
from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import HalvingGridSearchCV

grid = HalvingGridSearchCV(pipeline, param_grid, verbose = 10, scoring = 'f1_macro')
grid.fit(X_train, y_train)

n_iterations: 4
n_required_iterations: 6
n_possible_iterations: 4
min_resources_: 30
max_resources_: 860
aggressive_elimination: False
factor: 3
----------
iter: 0
n_candidates: 336
n_resources: 30
Fitting 5 folds for each of 336 candidates, totalling 1680 fits
[CV 1/5; 1/336] START clf=RandomForestClassifier(random_state=3380), clf__n_estimators=100, selection__percentile=20, vectorizer__bow__binary=True, vectorizer__bow__min_df=0.33, vectorizer__bow__ngram_range=(1, 1)
[CV 1/5; 1/336] END clf=RandomForestClassifier(random_state=3380), clf__n_estimators=100, selection__percentile=20, vectorizer__bow__binary=True, vectorizer__bow__min_df=0.33, vectorizer__bow__ngram_range=(1, 1);, score=(train=1.000, test=0.333) total time=   1.7s
[CV 2/5; 1/336] START clf=RandomForestClassifier(random_state=3380), clf__n_estimators=100, selection__percentile=20, vectorizer__bow__binary=True, vectorizer__bow__min_df=0.33, vectorizer__bow__ngram_range=(1, 1)
[CV 2/5; 1/336] END clf=RandomForestClassifie

/usr/local/lib/python3.7/dist-packages/sklearn/feature_selection/_univariate_selection.py:112: UserWarning: Features [  5   6  20  92 119 150 161 170] are constant.
  warnings.warn("Features %s are constant." % constant_features_idx, UserWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/feature_selection/_univariate_selection.py:113: RuntimeWarning: invalid value encountered in true_divide
  f = msb / msw


[CV 3/5; 1/336] END clf=RandomForestClassifier(random_state=3380), clf__n_estimators=100, selection__percentile=20, vectorizer__bow__binary=True, vectorizer__bow__min_df=0.33, vectorizer__bow__ngram_range=(1, 1);, score=(train=1.000, test=0.400) total time=   2.0s
[CV 4/5; 1/336] START clf=RandomForestClassifier(random_state=3380), clf__n_estimators=100, selection__percentile=20, vectorizer__bow__binary=True, vectorizer__bow__min_df=0.33, vectorizer__bow__ngram_range=(1, 1)


/usr/local/lib/python3.7/dist-packages/sklearn/feature_selection/_univariate_selection.py:112: UserWarning: Features [3 4] are constant.
  warnings.warn("Features %s are constant." % constant_features_idx, UserWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/feature_selection/_univariate_selection.py:113: RuntimeWarning: invalid value encountered in true_divide
  f = msb / msw


[CV 4/5; 1/336] END clf=RandomForestClassifier(random_state=3380), clf__n_estimators=100, selection__percentile=20, vectorizer__bow__binary=True, vectorizer__bow__min_df=0.33, vectorizer__bow__ngram_range=(1, 1);, score=(train=1.000, test=0.190) total time=   0.8s
[CV 5/5; 1/336] START clf=RandomForestClassifier(random_state=3380), clf__n_estimators=100, selection__percentile=20, vectorizer__bow__binary=True, vectorizer__bow__min_df=0.33, vectorizer__bow__ngram_range=(1, 1)


...

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


[CV 4/5; 13/13] END clf=LogisticRegression(random_state=3380), clf__penalty=none, selection__percentile=80, vectorizer__bow__binary=False, vectorizer__bow__min_df=1, vectorizer__bow__ngram_range=(1, 2);, score=(train=0.991, test=0.387) total time=  34.7s
[CV 5/5; 13/13] START clf=LogisticRegression(random_state=3380), clf__penalty=none, selection__percentile=80, vectorizer__bow__binary=False, vectorizer__bow__min_df=1, vectorizer__bow__ngram_range=(1, 2)


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


[CV 5/5; 13/13] END clf=LogisticRegression(random_state=3380), clf__penalty=none, selection__percentile=80, vectorizer__bow__binary=False, vectorizer__bow__min_df=1, vectorizer__bow__ngram_range=(1, 2);, score=(train=0.999, test=0.420) total time=  34.1s


HalvingGridSearchCV(estimator=Pipeline(steps=[('vectorizer',
                                               ColumnTransformer(transformers=[('bow',
                                                                                CountVectorizer(ngram_range=(1,
                                                                                                             2),
                                                                                                tokenizer=<__main__.StemmerTokenizer object at 0x7f3427c79990>),
                                                                                'history_text'),
                                                                               ('scaler',
                                                                                MinMaxScaler(),
                                                                                ['intelligence_score',
                                                                                

In [20]:
print(f"Best model: {grid.best_estimator_.get_params()['clf']}") # obtenemos el mejor modelo
print(f'Best score: {grid.best_score_}') # obtenemos el mejor score

Best model: LogisticRegression(penalty='none', random_state=3380)
Best score: 0.4679931972367548


In [23]:
grid.best_params_ # mejores parámetros

{'clf': LogisticRegression(penalty='none', random_state=3380),
 'clf__penalty': 'none',
 'selection__percentile': 20,
 'vectorizer__bow__binary': True,
 'vectorizer__bow__min_df': 1,
 'vectorizer__bow__ngram_range': (1, 2)}

In [21]:
# ranking de los modelos
ranking = pd.DataFrame(grid.cv_results_)
ranking = ranking.sort_values('rank_test_score')
ranking

,iter,n_resources,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_clf,param_clf__n_estimators,param_selection__percentile,param_vectorizer__bow__binary,...,mean_test_score,std_test_score,rank_test_score,split0_train_score,split1_train_score,split2_train_score,split3_train_score,split4_train_score,mean_train_score,std_train_score
451,2,270,8.394190,1.299804,1.282493,0.155609,"LogisticRegression(penalty='none', random_stat...",NaN,80,False,...,0.477237,0.113755,1,1.000000,1.000000,1.000000,0.990981,1.000000,0.998196,0.003608
489,3,810,18.020951,0.798944,3.860795,0.490358,"LogisticRegression(penalty='none', random_stat...",NaN,20,True,...,0.467993,0.026787,2,0.998779,0.998771,1.000000,0.998780,0.998787,0.999024,0.000488
330,0,30,0.600361,0.261056,0.252421,0.174624,"LogisticRegression(penalty='none', random_stat...",NaN,80,False,...,0.459481,0.202997,3,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000
493,3,810,15.862203,0.535265,3.708409,0.498709,"LogisticRegression(penalty='none', random_stat...",NaN,60,False,...,0.452555,0.053012,4,0.973624,0.954341,0.975830,0.978175,0.990734,0.974541,0.011713
129,0,30,0.473260,0.206604,0.169612,0.104552,SVC(random_state=3380),NaN,60,False,...,0.452222,0.215423,5,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,0,30,0.474523,0.196728,0.173956,0.109876,SVC(random_state=3380),NaN,20,True,...,0.215873,0.114568,495,0.844444,0.842415,1.000000,1.000000,1.000000,0.937372,0.076706
201,0,30,0.467350,0.202775,0.172451,0.105988,SVC(random_state=3380),NaN,20,False,...,0.212381,0.109735,496,0.496732,0.477778,0.794118,0.812500,0.908903,0.698006,0.176542
279,0,30,0.489850,0.207287,0.178483,0.110710,"LogisticRegression(penalty='none', random_stat...",NaN,80,True,...,0.211746,0.098493,497,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000
276,0,30,0.472093,0.193895,0.174564,0.107948,"LogisticRegression(penalty='none', random_stat...",NaN,80,True,...,0.194286,0.126577,498,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000


In [24]:
# print de classification report
y_pred = grid.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

         Bad       0.63      0.58      0.61       142
        Good       0.75      0.84      0.79       246
     Neutral       0.15      0.08      0.11        37

    accuracy                           0.69       425
   macro avg       0.51      0.50      0.50       425
weighted avg       0.66      0.69      0.67       425



Tras una laaaaaaaaaaarga espera (2 horas y media XD), se concluye que la mejor configuración para el problema la tiene la regresión logística sin regularización, seleccionando el top 20% de las features mas importantes, usando la versión binaria del Bag of Words, usando todas las palabras del vocabulario y sus respectivos bigramas. 

Como conclusión, se observa un mejor desempeño en todos los aspectos, comparando tanto con `MultinomialNB` como con el `DummyClassifier`. En otras palabras, el modelo seleccionado ofrece tanto un **Accuracy** global como mejores métricas de **Recall** por cada clase corrigiendo aunque no totalmente el problema de generar predicciones sesgadas a la clase mayoritaria.

## 1.4 Predicción del datos sin etiquetado  [0.5 puntos]

<p align="center">
  <img src="https://pbs.twimg.com/media/DolotxUUYAAbg7f.jpg" width="350">
</p>


LLego el momento de predecir 
`Vergil`, `Gorilla Girl` y `Batcow`


**Nota:** Recuerde que pueden existir campos vacios en `history_text`, por lo que se les recomienda borrar los nan.

**Respuesta:**

In [30]:
new_df = df_comics_no_label.dropna(subset = ['history_text']).copy().drop(columns = target) # filtro por valores nulos
new_df.head()

,Unnamed: 0,name,real_name,full_name,overall_score,history_text,powers_text,intelligence_score,strength_score,speed_score,...,has_flight,has_accelerated_healing,has_weapons_master,has_intelligence,has_reflexes,has_super_speed,has_durability,has_stamina,has_agility,has_super_strength
0,1,514A (Gotham),Bruce Wayne,NaN,10,He was one of the many prisoners of Indian Hil...,NaN,100,20,30,...,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0
2,17,A'dal,NaN,NaN,7,"As with most of the naaru, little is known of ...",NaN,85,30,70,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,27,Agent Zero (FOX),David North,NaN,6,"During mid-late 1973, Zero was a member of Tea...",Zero can absorb kinetic energy to further incr...,90,10,25,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
4,31,Ajax (FOX),Francis,Francis Freeman,7,Ajax (born Francis Freeman) was a human who ga...,Ajax has claimed that the procedure to obtain ...,85,25,45,...,0.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0
6,46,A.M.A.Z.O. (CW),NaN,NaN,22,The Anti Meta-human Adaptive Zootomic Organism...,"With its metallic body, it easily overpowered ...",90,100,100,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [32]:
new_df['prediction'] = grid.predict(new_df) # generamos predicciones sobre datos no vistos
new_df.head()

,Unnamed: 0,name,real_name,full_name,overall_score,history_text,powers_text,intelligence_score,strength_score,speed_score,...,has_accelerated_healing,has_weapons_master,has_intelligence,has_reflexes,has_super_speed,has_durability,has_stamina,has_agility,has_super_strength,prediction
0,1,514A (Gotham),Bruce Wayne,NaN,10,He was one of the many prisoners of Indian Hil...,NaN,100,20,30,...,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,Neutral
2,17,A'dal,NaN,NaN,7,"As with most of the naaru, little is known of ...",NaN,85,30,70,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Bad
3,27,Agent Zero (FOX),David North,NaN,6,"During mid-late 1973, Zero was a member of Tea...",Zero can absorb kinetic energy to further incr...,90,10,25,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,Good
4,31,Ajax (FOX),Francis,Francis Freeman,7,Ajax (born Francis Freeman) was a human who ga...,Ajax has claimed that the procedure to obtain ...,85,25,45,...,0.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,Good
6,46,A.M.A.Z.O. (CW),NaN,NaN,22,The Anti Meta-human Adaptive Zootomic Organism...,"With its metallic body, it easily overpowered ...",90,100,100,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Good


Finalmente, filtramos a los super héroes de los que queremos saber su predicción

In [41]:
to_predict = ['Vergil', 'Gorilla Girl', 'Batcow'] # lista de los super heroes
new_df[new_df['name'].isin(to_predict)][['name', 'history_text', 'prediction']] # filtro y visualizamos

,name,history_text,prediction
16,Batcow,Bat-Cow was originally a cow that was found by...,Bad
40,Gorilla Girl,A carnival performer with the ability to turn ...,Good
78,Vergil,"Vergil, later also known as Nelo Angelo, is on...",Neutral
83,Batcow,Bat-Cow was originally a cow that was found by...,Bad


<p align="center">
  <img src="https://media1.tenor.com/images/fb5bf7cc5a4acb91b4177672886a88ba/tenor.gif?itemid=5591338">
</p>

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=87110296-876e-426f-b91d-aaf681223468' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>